# Data Analysis using Pandas

### Question 1 (100 points) Celluloid ceiling

Wonder Woman             |  Captain Marvel
:-------------------------:|:-------------------------:
![wonderwoman](https://upload.wikimedia.org/wikipedia/en/e/ed/Wonder_Woman_%282017_film%29.jpg) | ![marvel](https://upload.wikimedia.org/wikipedia/pt/5/59/Captain_Marvel_%282018%29.jpg)

Women are involved in the film industry in all roles, including as film directors, actresses, cinematographers, film producers, film critics, and other film industry professions, though women have been underrepresented in all these positions. Studies found that women have always had a presence in film acting, but have consistently been underrepresented, and on average significantly less well paid. 

In 2015, Forbes reported that "...just 21 of the 100 top-grossing films of 2014 featured a female lead or co-lead, while only 28.1% of characters in 100 top-grossing films were female... This means it’s much rarer for women to get the sort of blockbuster role which would warrant the massive backend deals many male counterparts demand (Tom Cruise in Mission: Impossible or Robert Downey Jr. in Iron Man, for example)".

Also, Forbes' analysis of US acting salaries in 2013 determined that the "...men on Forbes’ list of top-paid actors for that year made 2½ times as much money as the top-paid actresses. That means that Hollywood's best-compensated actresses made just 40 cents for every dollar that the best-compensated men made. 


In this assignment, we want to examine whether and how women representation is lacking in the film industry. We will adopt The Bechdel test as a measure of the representation of women in the film industry. The test is named after the American cartoonist Alison Bechdel in whose 1985 comic strip Dykes to Watch Out For the test first appeared. **A movie is said to meet the Bechdel test  following three criteria: (1) it has to have at least two women in it, who (2) who talk to each other, about (3) something besides a man.**

We are going to obtain the data ourselves to perform the analysis. Specifically, we will retrieve the movie metadata from IMDB (Internet Movie Database), an online database of information related to films, television programs, home videos, video games, and streaming content online – including cast, production crew and personal biographies, plot summaries, trivia, ratings, and fan and critical reviews. As of January 2020, IMDb has approximately 6.5 million titles (including episodes) and 10.4 million personalities in its database, as well as 83 million registered users.


The IMDb Top 250 is a list of the top rated 250 films, based on ratings by the registered users of the website using the methods described. We will focus on these famous movies in this analysis:

**Question 1.1**: We will retrieve the metadata of IMDb Top 250 movies from the [IMDb charts](https://www.imdb.com/chart/top/). For each movie on the list, we can scrape the following characteristics from the information page. For example, from the [page of top rated movie "The Shawshank Redemption"](https://www.imdb.com/title/tt0111161/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=e31d89dd-322d-4646-8962-327b42fe94b1&pf_rd_r=F4QFC0SVZN1HTDHCY3C0&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_tt_1), we want to extract the metadata about this movie as:
- IMDb id (0111161)
- Movie name (The Shawshank Redemption)
- Year (1994)
- Director (Frank Darabont)
- Starring (Tim Robbins, Morgan Freeman, Bob Gunton)
- Rating (9.3)
- Number of reviews (2,291,324)
- Genres (Drama)
- Country (USA)
- Language (English)
- Budget (\$25,000,000)
- Box Office Revenue (\$28,815,291)
- Runtime (142 min)

![imdb](https://mrfloris.com/files/images/imdb-top250-page-start.png)

In [8]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [9]:
# Question 1.1

page = requests.get("https://www.imdb.com/chart/top/")
page

<Response [200]>

In [10]:
soup = BeautifulSoup(page.content, 'html.parser')

In [11]:
# We have found that the first title starts at line 59 of the code
tag = soup.find_all("a")[59]

# Find the 'href' that contains the link
link = tag['href']
link

'/title/tt0111161/'

In [12]:
link_dic = {}
rank = 0

# We add a step in the range since the links appear twince for the same title (image and url) and both go to the same place
for row in range(59, len(soup.find_all("a"))-54, 2):
    rank += 1
    temporary = soup.find_all("a")[row]
    link_dic[rank] = temporary['href']

In [13]:
half_url = link_dic[3]
page = requests.get("https://www.imdb.com/"+half_url)

In [14]:
new_soup = BeautifulSoup(page.content, 'html.parser')

## Scrapping IMDb website for the respective attributes

In [15]:
data = {}
rank = []
imdb_id = []
title = []
year = []
director = []
starring = []
rating = []
n_reviews = []
genre = []
country = []
box_off_rev = []
language = []
budget = []
runtime = []
for i in range(1,250+1):
    
    # Link to be scrapped
    half_url = link_dic[i]
    headers = {'Accept-Language': 'en-US, en;q=0.5'}
    page = requests.get("https://www.imdb.com/"+half_url, headers = headers)
    new_soup = BeautifulSoup(page.content, 'html.parser')
    
    # Rank
    rank.append(i)
    data['Rank'] = rank
    
    # IMDBid
    imdb_id.append(str(half_url[9:-1]))
    data['imdb_id'] = imdb_id
    
    # Movie Name
    t = new_soup.find(class_='title_wrapper').get_text().strip()
    tit = ''
    for i in t:
        if i == '\xa0':
            break
        tit+=i
    title.append(tit)
    data['Movie'] = title

    # Find the Year
    y = new_soup.find(attrs = {'id':'titleYear'}).get_text()
    year_1 = [i for i in y if i not in ['(',')']]
    year.append(''.join(year_1))
    data['Year'] = year

    # Find the Director
    d = new_soup.find_all('div', attrs = {'class':'credit_summary_item'})[0].get_text().split()
    director_1 = d[1:]
    director.append(' '.join(director_1))
    data['Director'] = director

    # Find Starring
    s = new_soup.find_all(class_ = 'credit_summary_item')[2].get_text().split()
    starring_1 = [i for i in s if i not in ['Stars:','|','See','full','cast','&','crew','»']]
    starring.append(' '.join(starring_1))
    data['Starring'] = starring
    
    # Finding the movie Rating
    rate = new_soup.find(attrs = {'itemprop':"ratingValue"}).get_text()
    rating.append(rate)
    data['Rating'] = rating

    # Number of reviews
    n_rev = new_soup.find(attrs = {'itemprop':'ratingCount'}).get_text()
    n_reviews.append(n_rev)
    data['#Reviews'] = n_reviews

    # Finding the Genre
    gen = new_soup.find_all('div', attrs = {'class':'inline'})[-1].get_text().strip().split()
    help_gen =[]
    for i in gen:
        if i not in ['Genres:','|']:
            help_gen.append(i)
    help_gen = ','.join(help_gen)
    genre.append(help_gen)
    data['Genre'] = genre

    # Finding Country and Language
    count = new_soup.find_all('div', attrs = {'class':'article', 'id':'titleDetails'})[0].get_text().split()
    counter = 0
    for i in count:
        counter+=1
        if i == 'Country:':
            if count[counter] == 'New':
                special = count[counter]+' '+count[counter+1]
                country.append(special)
            else:
                country.append(count[counter])
        if i == 'Language:':
            language.append(count[counter])
            break
        data['Country'] = country
        data['Language'] = language

    # Finding the Box Office Revenue
    try:
        Box_Off = new_soup.find_all('div', attrs = {'class':'txt-block'})[-7].get_text().split()[3]
        box_off_rev.append(Box_Off)
        data['Box_Off_Rev'] = box_off_rev
    except:
        box_off_rev.append(np.nan)
        data['Box_Off_Rev'] = box_off_rev
    
    # Finding the Budget
    a = new_soup.find_all('div', attrs = {'class':'txt-block'})[-10].get_text().strip()
    budget_list = a.split()[0]
    budget.append(budget_list[7:len(budget_list)])
    data['Budget'] = budget

    # Finding the Runtime
    run = new_soup.find_all('div', attrs = {'class':'txt-block'})[-4].get_text().strip()
    runtime_1 = run[9:]
    runt = ''
    for i in runtime_1:
        if i == '\n':
            break
        runt +=i
    runtime.append(runt)
    data['Runtime'] = runtime

new_data = pd.DataFrame(data)
new_data.to_csv('imdb_top_movies.csv')

In [16]:
new_data

,Rank,imdb_id,Movie,Year,Director,Starring,Rating,#Reviews,Genre,Country,Language,Box_Off_Rev,Budget,Runtime
0,1,0111161,The Shawshank Redemption,1994,Frank Darabont,"Tim Robbins, Morgan Freeman, Bob Gunton",9.3,"2,296,810",Drama,USA,English,"$28,815,291","$25,000,000",142 min
1,2,0068646,The Godfather,1972,Francis Ford Coppola,"Marlon Brando, Al Pacino, James Caan",9.2,"1,585,266","Crime,Drama",USA,English,"$246,120,986","$6,000,000",175 min
2,3,0071562,The Godfather: Part II,1974,Francis Ford Coppola,"Al Pacino, Robert De Niro, Robert Duvall",9.0,"1,107,591","Crime,Drama",USA,English,"$48,035,783","$13,000,000",202 min
3,4,0468569,The Dark Knight,2008,Christopher Nolan,"Christian Bale, Heath Ledger, Aaron Eckhart",9.0,"2,261,274","Action,Crime,Drama,Thriller",USA,English,"$1,005,456,758","$185,000,000",152 min
4,5,0050083,12 Angry Men,1957,Sidney Lumet,"Henry Fonda, Lee J. Cobb, Martin Balsam",8.9,"674,884","Crime,Drama",USA,English,$576,,96 min
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,246,0088247,The Terminator,1984,James Cameron,"Arnold Schwarzenegger, Linda Hamilton, Michael...",8.0,"789,639","Action,Sci-Fi",UK,English,"$78,680,331","$6,400,000",107 min
246,247,0103639,Aladdin,1992,"Ron Clements, John Musker","Scott Weinger, Robin Williams, Linda Larkin",8.0,"367,643","Animation,Adventure,Comedy,Family,Fantasy,Musi...",USA,English,"$504,050,219","$28,000,000",90 min
247,248,2991224,Tangerines,2013,Zaza Urushadze,"Lembit Ulfsak, Elmo Nüganen, Giorgi Nakashidze",8.2,"38,292","Drama,War",Estonia,Estonian,"$1,024,132","EUR650,000",87 min
248,249,5323662,A Silent Voice: The Movie,2016,Naoko Yamada,"Miyu Irino, Saori Hayami, Aoi Yûki",8.1,"42,824","Animation,Drama,Family,Romance",Japan,Japanese,"$30,490,447",,130 min


**Question 1.2** (5 points) If you group the movies by release years, show the number of movies at each decade in a descendingu order.

In [51]:
# Question 1.2
imdb_top_movies = pd.read_csv('imdb_top_movies.csv')

In [52]:
# Check the min Year and the max Year
# Define the Bins
# Get the data from the dataframe 
bins = [i for i in range(1920, 2020+1, 10)]
year_list = list(imdb_top_movies.Year)
decades = pd.cut(year_list, bins)

# Descending order of number of movies
pd.value_counts(decades)

(2010, 2020]    46
(2000, 2010]    46
(1990, 2000]    46
(1980, 1990]    26
(1970, 1980]    22
(1950, 1960]    22
(1960, 1970]    16
(1940, 1950]    11
(1930, 1940]     8
(1920, 1930]     7
dtype: int64

**Quesion 1.3** (5 points) Show the number of movies by the distribution of runtime at quartile (0-25%, 25-50%, 50-75%, 75-100%).

In [53]:
# Question 1.3
run_list = []
run = list(imdb_top_movies.Runtime)
for i in run:
    run_list.append(int(i[:-4]))

quartiles = pd.qcut(run_list, 4)  # Cut into quartiles

pd.value_counts(quartiles)

(44.999, 107.0]    64
(145.0, 321.0]     62
(126.0, 145.0]     62
(107.0, 126.0]     62
dtype: int64

**Question 1.4** (5 points) What is the proportion of movies that have Budget higher than 75% of all movies (i.e. the third quartile)?

In [54]:
# Question 1.4

# Get a clean data
not_null_budget = list(imdb_top_movies.Budget[imdb_top_movies.Budget.notnull()])
clean_budget = []
for i in not_null_budget:
    clean = []
    for j in i:
        if j in '1234567890':
            clean.append(j)
    if ''.join(clean) != '':
        clean_budget.append(int(''.join(clean)))
        
budget_quart = pd.qcut(clean_budget, 4) # Cut into quartiles
quart_list = [i for i in pd.value_counts(budget_quart)]
third_q = quart_list[3]/sum(quart_list)*100
print('{}% of movies have a Budget on the third quartile ( > 75% )'.format(round(third_q,2)))

23.46% of movies have a Budget on the third quartile ( > 75% )


**Question 1.5** (5 points) Show the top 10 most popular actor/actresses in terms of number of movies they have starred. 

In [55]:
# Question 1.5
from collections import Counter
stars_list = list(imdb_top_movies.Starring)
starring = [i.split(',') for i in stars_list]

starring_clean = []
for i in starring:
    for j in i:
        starring_clean.append(j)
        
# Remove the spacing that happends on the begining of the name in some names       
new_star = []
for i in range(1, len(starring_clean), 3):
    new_star.append(starring_clean[i][1:])
    
for i in range(2, len(starring_clean), 3):
    new_star.append(starring_clean[i][1:])
    
for i in range(0, len(starring_clean), 3):
    new_star.append(starring_clean[i])
    
# Check
# len(new_star) == len(starring_clean)

actors_n = dict(Counter(new_star))
pd.Series(actors_n).sort_values(ascending = False).head(10)

Robert De Niro       9
Harrison Ford        6
Leonardo DiCaprio    6
Charles Chaplin      6
Tom Hanks            6
Christian Bale       5
Clint Eastwood       5
Tatsuya Nakadai      4
Matt Damon           4
Brad Pitt            4
dtype: int64

**Question 1.6** (5 points) Show the top 5 directors with the most total box office revenues.

In [95]:
# Question 1.6
# Clean box office revenues
# Create a Series with the sum of box office revenues per director
# get the top 5

In [75]:
# Clean box office revenues
no_na_box = imdb_top_movies.Box_Off_Rev.dropna()
no_na_box = no_na_box.replace('Rodgers', np.nan)
no_na_box.str.replace(r'\D', '')
imdb_top_movies['Box_Off_Rev'] = no_na_box.str.replace(r'\D', '')
imdb_top_movies['Box_Off_Rev'] = imdb_top_movies['Box_Off_Rev'].replace('', np.nan)
imdb_top_movies['Box_Off_Rev'] = imdb_top_movies['Box_Off_Rev'].dropna()
try:
    imdb_top_movies['Box_Off_Rev'] = imdb_top_movies['Box_Off_Rev'].astype(float)
except:
    pass

In [57]:
imdb_top_movies.groupby(['Director'])['Box_Off_Rev'].sum().sort_values(ascending = False).head(5)

Director
Anthony Russo, Joe Russo    4.846160e+09
Christopher Nolan           4.143007e+09
Steven Spielberg            3.055116e+09
Peter Jackson               2.973971e+09
David Yates                 1.342207e+09
Name: Box_Off_Rev, dtype: float64

**Question 1.7** (5 points) Show the average ratings of movies across the genres and decades.

In [158]:
# Question 1.7
# Get the Decades Column
bins = [i for i in range(1920, 2020+1, 10)]
year_list = list(imdb_top_movies.Year)
decades = pd.cut(year_list, bins)
imdb_top_movies['decade'] = pd.Series(decades)

In [159]:
df = pd.DataFrame(imdb_top_movies.groupby(['Genre','decade'])['Rating'].mean().dropna())
df.head(25).sort_values('Rating', ascending = False)

Rating
Genre                               decade              
Action,Crime,Drama,Thriller         (2000, 2010]    9.00
Action,Adventure,Sci-Fi,Thriller    (2000, 2010]    8.80
Action,Adventure,Drama,Fantasy      (2000, 2010]    8.80
Action,Adventure,Fantasy,Sci-Fi     (1970, 1980]    8.65
Action,Drama,Mystery                (1960, 1970]    8.60
Action,Adventure,Drama              (1950, 1960]    8.60
                                    (1990, 2000]    8.50
Action,Crime,Drama,Thriller         (1990, 2000]    8.50
Action,Drama,Mystery,Thriller       (2000, 2010]    8.40
Action,Adventure                    (2010, 2020]    8.40
Action,Adventure,Drama,Sci-Fi       (2010, 2020]    8.40
Action,Adventure,Sci-Fi             (2010, 2020]    8.40
Action,Adventure                    (1980, 1990]    8.30
Action,Adventure,Sci-Fi,Thriller    (1980, 1990]    8.30
Action,Adventure,Fantasy,Sci-Fi     (1980, 1990]    8.30
Action,Biography,Drama,Sport        (2010, 2020]    8.20
Action,Adventure                    (2000, 2010]    8.20
Action,Comedy,Crime                 (1990, 2000]    8.20
Action,Comedy,Romance               (1920, 1930]    8.20
Action,Crime,Drama,Thriller         (2010, 2020]    8.20
Action,Adventure,Sci-Fi,Thriller    (2010, 2020]    8.10
Action,Crime,Drama,Mystery,Thriller (2000, 2010]    8.10
Action,Crime,Thriller               (2000, 2010]    8.10
Action,Adventure,Comedy,Drama,War   (1920, 1930]    8.10
Action,Adventure,Sci-Fi,Thriller    (1990, 2000]    8.10

In [160]:
# Create a new Dataframe with 
#  - Decades - Rating - Main Genre - Second ...
genre_list = list(imdb_top_movies.Genre)
genre = [i.split(',') for i in genre_list]     

genres_df = (imdb_top_movies['Genre'].str.split(',', expand=True).rename(columns=lambda x: f"genre_{x+1}"))
genres = pd.concat([genres_df, imdb_top_movies[['Rating','decade','Movie']]], axis = 1)
genres.rename(columns = {'genre_1':'Main'})

,Main,genre_2,genre_3,genre_4,genre_5,genre_6,genre_7,Rating,decade,Movie
0,Drama,None,None,None,None,None,None,9.3,"(1990, 2000]",The Shawshank Redemption
1,Crime,Drama,None,None,None,None,None,9.2,"(1970, 1980]",The Godfather
2,Crime,Drama,None,None,None,None,None,9.0,"(1970, 1980]",The Godfather: Part II
3,Action,Crime,Drama,Thriller,None,None,None,9.0,"(2000, 2010]",The Dark Knight
4,Crime,Drama,None,None,None,None,None,8.9,"(1950, 1960]",12 Angry Men
...,...,...,...,...,...,...,...,...,...,...
245,Action,Sci-Fi,None,None,None,None,None,8.0,"(1980, 1990]",The Terminator
246,Animation,Adventure,Comedy,Family,Fantasy,Musical,Romance,8.0,"(1990, 2000]",Aladdin
247,Drama,War,None,None,None,None,None,8.2,"(2010, 2020]",Tangerines
248,Animation,Drama,Family,Romance,None,None,None,8.1,"(2010, 2020]",A Silent Voice: The Movie


In [161]:
pd.DataFrame(genres.groupby(['genre_1','genre_2','genre_3','genre_4','genre_5','decade'])['Rating'].mean().dropna()).head(10).sort_values('Rating', ascending = False)

Rating
genre_1   genre_2   genre_3   genre_4  genre_5  decade              
Animation Adventure Drama     Family   Musical  (1990, 2000]    8.50
          Action    Adventure Family   Sci-Fi   (2010, 2020]    8.40
Adventure Biography Drama     History  War      (1960, 1970]    8.30
          Drama     History   Thriller War      (1960, 1970]    8.20
Animation Adventure Comedy    Family   Fantasy  (1990, 2000]    8.15
                                                (2000, 2010]    8.15
Action    Adventure Comedy    Drama    War      (1920, 1930]    8.10
          Crime     Drama     Mystery  Thriller (2000, 2010]    8.10
Animation Action    Adventure Family   Fantasy  (2000, 2010]    8.10
          Adventure Comedy    Drama    Family   (2010, 2020]    8.10

In [162]:
imdb_top_movies.Genre
gen = imdb_top_movies['Genre'].str.get_dummies(',')
gen['Rank'] = imdb_top_movies.Rank

In [163]:
genres = pd.merge(imdb_top_movies[['decade','Rating','Rank']], gen, on = 'Rank')

In [164]:
genres_dec = list(genres.columns[3:])
genres_dec.append('decade')

In [165]:
genres

,decade,Rating,Rank,Action,Adventure,Animation,Biography,Comedy,Crime,Drama,...,Horror,Music,Musical,Mystery,Romance,Sci-Fi,Sport,Thriller,War,Western
0,"(1990, 2000]",9.3,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,"(1970, 1980]",9.2,2,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
2,"(1970, 1980]",9.0,3,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
3,"(2000, 2010]",9.0,4,1,0,0,0,0,1,1,...,0,0,0,0,0,0,0,1,0,0
4,"(1950, 1960]",8.9,5,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,"(1980, 1990]",8.0,246,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
246,"(1990, 2000]",8.0,247,0,1,1,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
247,"(2010, 2020]",8.2,248,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
248,"(2010, 2020]",8.1,249,0,0,1,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0


In [224]:
genre_columns = list(genres.columns[3:])
for i in genre_columns:
    genres[i] = genres[i].replace(1, i)
genres = genres.replace(0, np.nan)
genres

,decade,Rating,Rank,Action,Adventure,Animation,Biography,Comedy,Crime,Drama,...,Horror,Music,Musical,Mystery,Romance,Sci-Fi,Sport,Thriller,War,Western
0,"(1990, 2000]",9.3,1,NaN,NaN,NaN,NaN,NaN,NaN,Drama,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"(1970, 1980]",9.2,2,NaN,NaN,NaN,NaN,NaN,Crime,Drama,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"(1970, 1980]",9.0,3,NaN,NaN,NaN,NaN,NaN,Crime,Drama,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"(2000, 2010]",9.0,4,Action,NaN,NaN,NaN,NaN,Crime,Drama,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Thriller,NaN,NaN
4,"(1950, 1960]",8.9,5,NaN,NaN,NaN,NaN,NaN,Crime,Drama,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,"(1980, 1990]",8.0,246,Action,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Sci-Fi,NaN,NaN,NaN,NaN
246,"(1990, 2000]",8.0,247,NaN,Adventure,Animation,NaN,Comedy,NaN,NaN,...,NaN,NaN,Musical,NaN,Romance,NaN,NaN,NaN,NaN,NaN
247,"(2010, 2020]",8.2,248,NaN,NaN,NaN,NaN,NaN,NaN,Drama,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,War,NaN
248,"(2010, 2020]",8.1,249,NaN,NaN,Animation,NaN,NaN,NaN,Drama,...,NaN,NaN,NaN,NaN,Romance,NaN,NaN,NaN,NaN,NaN


In [225]:
# User Input of the genre
genre_columns = list(genres.columns[3:])
genre_input = 0
while genre_input not in genre_columns:
    print("""The gender list is:
{}
    """.format(genre_columns))
    genre_input = str(input('Please select the genre you want to see the average ratings over the decades: '))
    
genres_mean_rat = pd.DataFrame(genres.groupby([genre_input,'decade'])['Rating'].mean())
genres_mean_rat.sort_values('decade', ascending = False)

The gender list is:
['Action', 'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime', 'Drama', 'Family', 'Fantasy', 'Film-Noir', 'History', 'Horror', 'Music', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Sport', 'Thriller', 'War', 'Western']
    
Please select the gender you want to see the average ratings over the decades: Comedy


Rating
Comedy decade                
Comedy (2010, 2020]  8.250000
       (2000, 2010]  8.200000
       (1990, 2000]  8.228571
       (1980, 1990]  8.300000
       (1970, 1980]  8.200000
       (1960, 1970]  8.400000
       (1950, 1960]  8.266667
       (1940, 1950]  8.200000
       (1930, 1940]  8.320000
       (1920, 1930]  8.180000

**Question 1.8** (5 points) Creat a new column ```ROI``` that measures the return on investment using the (box revenue-budget)/budget, and compare the ROI between movies in English and those in non-English. Use the t-test to examine whether such difference is statistically significant (You can use ```scipy.stats.ttest_ind``` to test the mean difference of two distributions)

In [76]:
# Clean the Budget:
budget_clean = imdb_top_movies.Budget
budget_clean = budget_clean.str.replace(r'\D', '')
budget_clean = budget_clean.replace('', np.nan)
try:
    budget_clean = budget_clean.astype(float)
except:
    pass
imdb_top_movies['Budget'] = budget_clean

In [78]:
# Question 1.8
imdb_top_movies['ROI (%)'] = ((imdb_top_movies.Box_Off_Rev - imdb_top_movies.Budget)/(imdb_top_movies.Budget))*100
roi_eng = pd.DataFrame(imdb_top_movies.groupby(imdb_top_movies['Language'] == 'English')['ROI (%)'].mean())
roi_eng

,ROI (%)
Language,
False,732.543978
True,720.849294


In [79]:
roi_eng = np.array(imdb_top_movies.loc[imdb_top_movies['Language']=='English']['ROI (%)'].dropna())
roi_not_eng = np.array(imdb_top_movies.loc[imdb_top_movies['Language'] != 'English']['ROI (%)'].dropna())

In [80]:
from scipy import stats
stats.ttest_ind(roi_eng, roi_not_eng)

Ttest_indResult(statistic=-0.055373063256047884, pvalue=0.9559037092049489)

In [81]:
print('''Therefore we can conclude that the difference between English versus Non-English movies is not statistically significant given the very high p_value''')

Therefore we can conclude that the difference between English versus Non-English movies is not statistically significant given the very high p_value


**Question 1.9** (5 points) Do the commercially successfuly movies also receive higher ratings. Check the correlations between box office revenues and ratings using Pearman and Spearman correlations.

In [82]:
# Use groupby and aggregate because agregate allows for different operations LEcture 11, slide 20
# Or use apply 
pearson_corr = imdb_top_movies[['Rating', 'Box_Off_Rev']]
pearson = pearson_corr.Box_Off_Rev.corr(pearson_corr.Rating, method = 'pearson')
spearman_corr = imdb_top_movies[['Rating', 'Box_Off_Rev']]
spearman = spearman_corr.Box_Off_Rev.corr(spearman_corr.Rating, method = 'spearman')

print('The pearson coefficient is {} and the spearman coefficient is {}'.format(pearson, spearman))
print('''\nBoth correlation values are relatively small and close to 0, therefore there is a positive correlation but it is small, 
implying only a weak support for the claim that commercialy successeful movies tend to receive higher ratings''')
print('''\nPearson correlation assumes the data is normally distributed. However, Spearman does not make any assumption on the 
distribution of the data. That is the main reason for the difference.''')

The pearson coefficient is 0.2083292592490351 and the spearman coefficient is 0.1468032682570735

Both correlation values are relatively small and close to 0, therefore there is a positive correlation but it is small, 
implying only a weak support for the claim that commercialy successeful movies tend to receive higher ratings

Pearson correlation assumes the data is normally distributed. However, Spearman does not make any assumption on the 
distribution of the data. That is the main reason for the difference.


In [83]:
imdb_top_movies

,Unnamed: 0,Rank,imdb_id,Movie,Year,Director,Starring,Rating,#Reviews,Genre,Country,Language,Box_Off_Rev,Budget,Runtime,ROI (%)
0,0,1,111161,The Shawshank Redemption,1994,Frank Darabont,"Tim Robbins, Morgan Freeman, Bob Gunton",9.3,"2,296,810",Drama,USA,English,2.881529e+07,25000000.0,142 min,15.261164
1,1,2,68646,The Godfather,1972,Francis Ford Coppola,"Marlon Brando, Al Pacino, James Caan",9.2,"1,585,266","Crime,Drama",USA,English,2.461210e+08,6000000.0,175 min,4002.016433
2,2,3,71562,The Godfather: Part II,1974,Francis Ford Coppola,"Al Pacino, Robert De Niro, Robert Duvall",9.0,"1,107,591","Crime,Drama",USA,English,4.803578e+07,13000000.0,202 min,269.506023
3,3,4,468569,The Dark Knight,2008,Christopher Nolan,"Christian Bale, Heath Ledger, Aaron Eckhart",9.0,"2,261,274","Action,Crime,Drama,Thriller",USA,English,1.005457e+09,185000000.0,152 min,443.490139
4,4,5,50083,12 Angry Men,1957,Sidney Lumet,"Henry Fonda, Lee J. Cobb, Martin Balsam",8.9,"674,884","Crime,Drama",USA,English,5.760000e+02,NaN,96 min,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,245,246,88247,The Terminator,1984,James Cameron,"Arnold Schwarzenegger, Linda Hamilton, Michael...",8.0,"789,639","Action,Sci-Fi",UK,English,7.868033e+07,6400000.0,107 min,1129.380172
246,246,247,103639,Aladdin,1992,"Ron Clements, John Musker","Scott Weinger, Robin Williams, Linda Larkin",8.0,"367,643","Animation,Adventure,Comedy,Family,Fantasy,Musi...",USA,English,5.040502e+08,28000000.0,90 min,1700.179354
247,247,248,2991224,Tangerines,2013,Zaza Urushadze,"Lembit Ulfsak, Elmo Nüganen, Giorgi Nakashidze",8.2,"38,292","Drama,War",Estonia,Estonian,1.024132e+06,650000.0,87 min,57.558769
248,248,249,5323662,A Silent Voice: The Movie,2016,Naoko Yamada,"Miyu Irino, Saori Hayami, Aoi Yûki",8.1,"42,824","Animation,Drama,Family,Romance",Japan,Japanese,3.049045e+07,NaN,130 min,NaN


**Question 1.10** (10 points) Now let's retrieve data from Bechdel Test Movie website [for each movie](https://bechdeltest.com/). You can send the requests to the API: https://bechdeltest.com/api/v1/doc#getMovieByImdbId. For example, for the movie The Shawshank Redemption (the IMDb id: 0111161), you can simply call: http://bechdeltest.com/api/v1/getMovieByImdbId?imdbid=0111161. 

Create a dataframe ```bechdel_imdb_top``` that merge the bechdel test info with the ```imdb_top_movies``` show how many top 250 movies are also in the bechdel test website.

In [ ]:
# I have kept the imdb_top_movies.csv file as originally scrapped from the website
# and chose to keep the changes dependend on running the whole data analysis above, therefore, columns created above
# such as ROI (%) and columns that were cleaned, are necessary for the next analysis but that code needs to be run
# first, to get the columns and the most updated/cleaned version of the imdb_top_movies dataframe.

In [84]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

bechdel_list = []
for i in range(1, 250+1):
    imdb_id = link_dic[i][9:-1]
    bechdel = requests.get("http://bechdeltest.com/api/v1/getMovieByImdbId?imdbid="+str(imdb_id))
    bechdel = bechdel.json()
    bechdel_list.append(bechdel)

In [85]:
bechdel_imdb_aux = pd.DataFrame(bechdel_list)

In [86]:
bechdel_imdb_aux['Rank'] = imdb_top_movies.Rank

In [87]:
bechdel_imdb_top = pd.merge(bechdel_imdb_aux, imdb_top_movies, on = 'Rank')

In [88]:
bechdel_imdb_top.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 250 entries, 0 to 249
Data columns (total 28 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           235 non-null    float64
 1   submitterid  235 non-null    float64
 2   visible      235 non-null    object 
 3   rating       235 non-null    float64
 4   year         235 non-null    float64
 5   date         235 non-null    object 
 6   dubious      199 non-null    object 
 7   imdbid       235 non-null    object 
 8   title        235 non-null    object 
 9   version      15 non-null     object 
 10  status       15 non-null     object 
 11  description  15 non-null     object 
 12  Rank         250 non-null    int64  
 13  Unnamed: 0   250 non-null    int64  
 14  imdb_id      250 non-null    int64  
 15  Movie        250 non-null    object 
 16  Year         250 non-null    int64  
 17  Director     250 non-null    object 
 18  Starring     250 non-null    object 
 19  Rating  

In [97]:
visible = bechdel_imdb_top.visible.count()

In [98]:
print('In the Bechdel Website there are {} movies that are also on the top 250 of imdb.'.format(visible))

In the Bechdel Website there are 235 movies that are also on the top 250 of imdb.


**Question 1.11** (5 points) Show how many movies in terms of percentage) that has passed the test in different ways (Number from 0 to 3 (0 means no two women, 1 means no talking, 2 means talking about a man, 3 means it passes the test)

In [100]:
pass_test_percent = bechdel_imdb_top.groupby('rating')['Movie'].count()
pass_test_percent.apply(lambda x: (x/pass_test_percent.sum())*100)

rating
0.0    20.851064
1.0    34.468085
2.0     9.787234
3.0    34.893617
Name: Movie, dtype: float64

**Question 1.12** (5 points) Show the percenage of movies given differen genres that has passed the test in different ways (Number from 0 to 3 (0 means no two women, 1 means no talking, 2 means talking about a man, 3 means it passes the test))

In [261]:
pass_test_percent_genre = pd.DataFrame(bechdel_imdb_top.groupby(['rating','Genre'])['Movie'].count())
pass_test_percent_genre.Movie.apply(lambda x: (x/pass_test_percent_genre.sum())*100).sort_values('rating', ascending = False)

Movie
rating Genre                                    
3.0    Mystery,Thriller                 0.425532
       Adventure,Drama,History          0.425532
       Biography,Crime,Drama            0.851064
       Animation,Family,Fantasy         0.425532
       Animation,Drama,Fantasy,Romance  0.425532
...                                          ...
0.0    Crime,Drama,Mystery              0.425532
       Crime,Drama                      1.276596
       Comedy,War                       0.425532
       Comedy,Romance                   0.425532
       Action,Adventure                 0.425532

[169 rows x 1 columns]

In [260]:
genre_columns = list(genres.columns[3:])
genre_input = 0
while genre_input not in genre_columns:
    print("""The gender list is:
{}
    """.format(genre_columns))
    genre_input = str(input('Please select the genre you want to see the % of movies that pass each  test degree (0,1,2,3): '))
    
# Total Number of Titles in the Bechdel dataset that are in Imdb top 250
total = bechdel_imdb_top.title.count()

pass_genre_percent = pd.merge(genres, bechdel_imdb_top, on='Rank')
pass_genre_percent = pd.DataFrame(pass_genre_percent.groupby([genre_input,'rating'])['Movie'].count())
pass_genre_percent = pd.DataFrame(pass_genre_percent.Movie.apply(lambda x: (x/total)*100)).sort_values('rating', ascending = False)
pass_genre_percent.rename(columns = {'Movie':'Movie (%)'})

The gender list is:
['Action', 'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime', 'Drama', 'Family', 'Fantasy', 'Film-Noir', 'History', 'Horror', 'Music', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Sport', 'Thriller', 'War', 'Western']
    
Please select the genre you want to see the % of movies that pass each  test degree (0,1,2,3): Action


Movie (%)
Action rating           
Action 3.0      8.510638
       1.0      5.957447
       0.0      2.978723

**Question 1.13** (5 points) Show the top 10 highest-rated movies that passed the test completely (rating=3) 

In [262]:
bechdel_imdb_top.loc[bechdel_imdb_top['rating']==3.0].sort_values(['Rating'], ascending = False)[['Movie','Rating']].head(10)

,Movie,Rating
3,The Dark Knight,9.0
7,Pulp Fiction,8.9
5,Schindler's List,8.9
12,Inception,8.8
13,The Lord of the Rings: The Two Towers,8.7
15,The Matrix,8.7
16,Goodfellas,8.7
26,Spirited Away,8.6
29,Interstellar,8.6
28,Parasite,8.6


**Question 1.14** (5 points) Compareing the movies that passed (rating=3) and failed the test (rating=0), are their ROI different? Explain.

In [263]:
rat_3 = bechdel_imdb_top.loc[bechdel_imdb_top['rating']==3.0]['ROI (%)'].mean()
rat_0 = bechdel_imdb_top.loc[bechdel_imdb_top['rating']==0.0]['ROI (%)'].mean()
pd.DataFrame({'rating = 3': [rat_3], 'rating = 0': [rat_0]}, index = ['ROI %'])

,rating = 3,rating = 0
ROI %,846.749748,617.732173


In [264]:
print('''Return On Investment (ROI) seems to be significantly higher on average for movies that pass the Bechdel Test
''')

Return On Investment (ROI) seems to be significantly higher on average for movies that pass the Bechdel Test



**Question 1.15** (10 points) Now load the ```bechdel_imdb.json``` that contains the all movies that are rated by the Bechdel Test website. Are women representation improved over the decades? Create a dataframe ```bechdel_imdb```, comparing the top 250 and other movies, in terms of percentage, how many passed/failed the test? 

In [265]:
with open('bechdel_imdb.json') as json_file:
    bechdel_imdb = pd.read_json(json_file)

In [266]:
all_data_pass_rate = bechdel_imdb.groupby('rating')['title'].count()
a = pd.DataFrame(all_data_pass_rate.apply(lambda x: (x/all_data_pass_rate.sum())*100))
a.rename(columns = {'title': 'Pass rate (%)'})

,Pass rate (%)
rating,
0,10.158619
1,21.950082
2,10.181945
3,57.709354


In [267]:
# Get the min and the max year
print('Min year,', bechdel_imdb.year.min())
print('Max year,', bechdel_imdb.year.max())

Min year, 1888
Max year, 2020


In [268]:
bins = [i for i in range(1880, 2020+1, 10)]
year_list_bech = list(bechdel_imdb.year)
decades_bech = pd.cut(year_list_bech, bins)

bechdel_imdb['decade'] = pd.Series(decades_bech)

In [269]:
bechdel_imdb

,year,imdbid,rating,title,id,decade
0,1888,0392728,0,Roundhay Garden Scene,8040,"(1880, 1890]"
1,1892,0000003,0,Pauvre Pierrot,5433,"(1890, 1900]"
2,1895,0132134,0,"Execution of Mary, Queen of Scots, The",6200,"(1890, 1900]"
3,1895,0000014,0,Tables Turned on the Gardener,5444,"(1890, 1900]"
4,1896,0000131,0,Une nuit terrible,5406,"(1890, 1900]"
...,...,...,...,...,...,...
8569,2020,7134096,2,"Rhythm Section, The",8994,"(2010, 2020]"
8570,2020,8461042,3,"Marijuana Conspiracy , The",8859,"(2010, 2020]"
8571,2020,1502397,2,Bad Boys For Life,9071,"(2010, 2020]"
8572,2020,7713068,3,Birds of Prey,9008,"(2010, 2020]"


In [270]:
pass_test_decades = bechdel_imdb.loc[bechdel_imdb['rating']==3.0].sort_values(['rating'], ascending = False)[['decade','rating','title']]
pass_test_decades.groupby('decade').count()

,rating,title
decade,,
"(1880, 1890]",0,0
"(1890, 1900]",1,1
"(1900, 1910]",1,1
"(1910, 1920]",12,12
"(1920, 1930]",21,21
"(1930, 1940]",107,107
"(1940, 1950]",104,104
"(1950, 1960]",139,139
"(1960, 1970]",152,152


In [271]:
print('As we can observe in the table above, female representation has increased over the decades')

As we can observe in the table above, female representation has increased over the decades
